# Introdução


A Rossmann, com mais de 3.000 farmácias em sete países europeus, está encarando um desafio e tanto: prever as vendas diárias com até seis semanas de antecedência. Nossos gerentes de loja estão no front dessa missão, considerando fatores como promoções, competição, feriados escolares, sazonalidade e localidade. O detalhe é que cada gerente prevê com base em suas próprias situações, o que gera uma certa variação nos resultados.

Neste contexto, a precisão das previsões de vendas pode variar significativamente, uma vez que cada gerente prediz as vendas com base em suas circunstâncias únicas. Com o intuito de aprimorar esse processo, a Rossmann lançou seu primeiro desafio no Kaggle, convocando profissionais de Data Science para desenvolverem modelos preditivos robustos.

## Ojbetivo principal

- Aqui, enfrentamos o desafio de prever com precisão as vendas diárias de mais de 1.100 lojas da Rossmann na Alemanha, ao longo de um período de seis semanas. Por quê? Porque essas previsões são cruciais para que os gerentes de loja possam elaborar escalas de trabalho eficientes, o que, por sua vez, impulsiona a produtividade e a motivação das equipes.

- A avaliação do desempenho dos modelos será feita por meio da métrica Root Mean Square Percentage Error (RMSPE). Essa métrica garante que as previsões estejam o mais próximas possível das vendas reais, com uma penalização mais significativa para erros percentuais maiores. Importante notar que durante a avaliação, dias e lojas sem vendas serão excluídos, alinhando-se à operação prática da Rossmann.

In [21]:
## Helper Functions

## DATA

In [1]:
# Caminho dos arquivos
caminho_store = r'C:\Users\daler\Downloads\rossmann-store-sales\store.csv'
caminho_train = r'C:\Users\daler\Downloads\rossmann-store-sales\train.csv'
caminho_test = r'C:\Users\daler\Downloads\rossmann-store-sales\test.csv'

In [31]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import math
import inflection

# Desativar todos os warnings

In [4]:
df_sales = pd.read_csv(caminho_train, low_memory=False, encoding='utf-8')
df_store = pd.read_csv(caminho_store, low_memory=False, encoding='utf-8')

In [5]:
# merge
df = pd.merge(df_sales, df_store, how='left', on='Store')

In [6]:
df.head(2)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"


In [79]:
# Fazendo backup do dataframe
df1 = df.copy()

### Mudança de atributos

In [80]:
df.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval'],
      dtype='object')

In [81]:
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday',
           'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth',
          'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']
snakecase = lambda x: inflection.underscore(x)

cols_new = list(map(snakecase, cols_old))

# Renomenado

df1.columns = cols_new

In [82]:
df1.shape

(1017209, 18)

In [83]:
df1.dtypes

store                             int64
day_of_week                       int64
date                             object
sales                             int64
customers                         int64
open                              int64
promo                             int64
state_holiday                    object
school_holiday                    int64
store_type                       object
assortment                       object
competition_distance            float64
competition_open_since_month    float64
competition_open_since_year     float64
promo2                            int64
promo2_since_week               float64
promo2_since_year               float64
promo_interval                   object
dtype: object

In [84]:
df1.shape

(1017209, 18)

In [85]:
df1.dtypes

store                             int64
day_of_week                       int64
date                             object
sales                             int64
customers                         int64
open                              int64
promo                             int64
state_holiday                    object
school_holiday                    int64
store_type                       object
assortment                       object
competition_distance            float64
competition_open_since_month    float64
competition_open_since_year     float64
promo2                            int64
promo2_since_week               float64
promo2_since_year               float64
promo_interval                   object
dtype: object

- Algumas variáveis estão com formato (tipo) de dados não adequados corretamente. A seguir será feita uma correção dos tipos desses dados

Para realizar a mudança dos tipos de dados, será utilizado modulos importados

Os módulos a seguir foram criados durante o curso de Formação Analista de Dados (FADA) da Data Science Academy com auxilio do professor em video aula

Os módulos se tratam de técnicas de tratamento de dados que podem ser reutilizadas em diferentes casos e análises, sendo necessária somente a importação do mesmo

A seguir segue a descrição de cada módulo

valores_ausentes.py -
- identifica a porcentagem de valores ausentes
- diversas funções com técnicas de tratamentos dos valores ausentes, sendo elas:
    - imputação (bfill, ffill, media e mediana)
    - drop de linhas ou colunas

conversao.py - Modulos de conversão dos tipos de dados para
- data
- string
- int64
- fator

outliers.py -
- identifica a porcentagem de outliers
- realiza o tratamento desses outliers (podendo ser o drop/exclusão dos mesmos, ou replace/modificação)


In [86]:
from tratamento_dados.conversao import *
from tratamento_dados.outliers import *
from tratamento_dados.valores_ausentes import *

In [87]:
# Convertendo a coluna data para o tipo datetime
convert_to_datetime(df1, ['date'])

In [88]:
func_calc_percentual_valores_ausentes(df1)

O dataset tem 11.87 % de valores ausentes.


In [ ]:
func_calc_percentual_valores_ausentes_linha(df1)

In [90]:
func_calc_percentual_valores_ausentes_coluna(df1)

O dataset tem 18 colunas.
Encontrado: 6 colunas que têm valores ausentes.


,Valores Ausentes,% de Valores Ausentes,Dtype
promo2_since_week,508031,49.940,float64
promo2_since_year,508031,49.940,float64
promo_interval,508031,49.940,object
competition_open_since_month,323348,31.790,float64
competition_open_since_year,323348,31.790,float64
competition_distance,2642,0.260,float64


### Preenchendo valores ausentes

#### competition_distance 

competition_distance: distance in meters to the nearest competitor store

Essa coluna descreve a distancia em metros do competidor mais próximo da loja

In [91]:
# Configuração para exibir números sem notação científica
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [92]:
df1.competition_distance.round(2).describe()

count   1014567.000
mean       5430.086
std        7715.324
min          20.000
25%         710.000
50%        2330.000
75%        6890.000
max       75860.000
Name: competition_distance, dtype: float64

- Visualizando a descrição da coluna competition_distance nota-se que não há valores 0 ou -1 (que poderia indicar que não há concorrentes próximos para essa loja). 

- Adotando que os valores ausentes para os concorrentes são lojas onde não há concorrencia, iremos imputar o valor máximo de distancia encontrado nesta coluna * 2, tentando manter a possivel similaridade entre a movimentação das lojas que não possuem concorrentes, com aquelas que as lojas possuem concorrentes mas eles estão muitos distantes que devem não impactar tanto na loja

In [93]:
# tratando competition_distance - distance in meters to the nearest competitor store

max_value = df1.competition_distance.max()

df1['competition_distance'] = df1['competition_distance'].apply(lambda x: (max_value * 2) if math.isnan(x) else x)

In [94]:
df1['competition_distance'][df1['competition_distance'] > 50000].value_counts() 
# df1.competition_distance.value_counts().nlargest(20) 

competition_distance
151720.000    2642
58260.000      942
75860.000      942
Name: count, dtype: int64

In [95]:
df1.competition_distance.describe()

count   1017209.000
mean       5810.045
std       10715.013
min          20.000
25%         710.000
50%        2330.000
75%        6910.000
max      151720.000
Name: competition_distance, dtype: float64

Apesar de impactar um pouco com a coluna competition_distance, a depender do cenário para o qual utilizaremos essas medidas, podemos tratar os outliers afim de minimizar os danos das alterações realizadas.

- competition_open_since_month - gives the approximate year and month of the time the nearest competitor was opened > data aproximada da abertura da loja do competidor mais próximo

- Podemos supor que os valores ausentes nesses dados são por 2 motivos:
    - Não há concorrencia próxima.
    - No fechamento do dia, ainda não havia concorrencia próxima.
    - A loja se instalou depois da concorrencia.

- Embora há uma quantidade consideravel de valores ausentes (31%) acredito que essa coluna seja importante para a análise de vendas. A fixação de uma loja concorrente pode afetar de forma agressiva as vendas de uma loja durante um periodo até a estabilização. 

- Para isso, iremos imputar a data de fechamento do dia

In [97]:
df1['date'] = pd.to_datetime(df1['date'], errors='coerce')

# Preenchendo os valores nulos em 'competition_open_since_month' com o mês correspondente em 'date'
df1['competition_open_since_month'].fillna(df1['date'].dt.month, inplace=True)

In [101]:
# Preenchendo os valores nulos em 'competition_open_since_year' com o mês correspondente em 'date'
df1['competition_open_since_year'].fillna(df1['date'].dt.year, inplace=True)

In [102]:
func_calc_percentual_valores_ausentes_coluna(df1)

O dataset tem 18 colunas.
Encontrado: 3 colunas que têm valores ausentes.


,Valores Ausentes,% de Valores Ausentes,Dtype
promo2_since_week,508031,49.940,float64
promo2_since_year,508031,49.940,float64
promo_interval,508031,49.940,object


- Promo2 - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating

- Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2

- Será imputado o dia do fechamento, supondo que os valores ausentes são quando as promoções não continuaram (identificado pela coluna Promo2=0, portanto para aqueles que continuaram contem os dados, e para tratar os ausentes sem ter que excluir a coluna imputaremos o dia do fechamento)


In [104]:
df1['promo2_since_week'].fillna(df1['date'].dt.isocalendar().week, inplace=True)
df1['promo2_since_year'].fillna(df1['date'].dt.year, inplace=True)

- promo_interval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store

- Vamos criar uma coluna de apoio para que possamos verificar se o mês está dentro do intervalo de promoção, e depois criar uma nova coluna identificando se está ocorrendo promoção ou não

In [105]:
month_map = {1: "Jan", 2: "Feb", 3: "Mar", 4: "Apr", 5: "May", 6: "Jun", 7: "Jul", 8: "Aug", 9: "Sep", 10: "Oct", 11: "Nov", 12: "Dec"}
df1['promo_interval'].fillna(0, inplace=True)

In [106]:
df1['month_map'] = df1['date'].dt.month.map(month_map)

In [107]:
df1.sample(5).T

,594928,1016926,687543,226946,29989
store,304,832,374,602,1000
day_of_week,2,2,3,5,7
date,2014-01-14 00:00:00,2013-01-01 00:00:00,2013-10-23 00:00:00,2015-01-09 00:00:00,2015-07-05 00:00:00
sales,6461,0,6797,6647,0
customers,1066,0,863,605,0
open,1,0,1,1,0
promo,0,0,1,1,0
state_holiday,0,a,0,0,0
school_holiday,0,1,0,1,0
store_type,a,d,a,a,a


In [108]:
df1['is_promo'] =  df1[['promo_interval', 'month_map']].apply(lambda x: 0 if x['promo_interval'] == 0 else 1 if x['month_map'] in x['promo_interval'].split( ',' ) else 0, axis=1)

In [109]:
df1.sample(7).T

,5347,979523,535209,205994,1005347,526367,118076
store,888,224,795,835,403,873,1002
day_of_week,1,7,7,3,5,1,5
date,2015-07-27 00:00:00,2013-02-03 00:00:00,2014-03-09 00:00:00,2015-01-28 00:00:00,2013-01-11 00:00:00,2014-03-17 00:00:00,2015-04-17 00:00:00
sales,12303,0,0,5199,8879,5189,7132
customers,910,0,0,621,821,566,853
open,1,0,0,1,1,1,1
promo,1,0,0,1,1,1,1
state_holiday,0,0,0,0,0,0,0
school_holiday,1,0,0,0,0,0,0
store_type,d,d,d,a,a,a,d


In [110]:
func_calc_percentual_valores_ausentes_coluna(df1)

O dataset tem 20 colunas.
Encontrado: 0 colunas que têm valores ausentes.
